In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Importing libraries**

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC


In [4]:
import nltk # 문장 토크나이저

# 영어 불용어 - 불용어 모아 놓은 리스트 다운로드해 제거
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize # 토큰화
from nltk.stem.porter import PorterStemmer # 어근 동일화 <-> 이거 말고도 "Lancaster Stemmer"

# 표제어 추출
from nltk.stem import WordNetLemmatizer

# 정규표현 처리
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [5]:
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test_jm.csv')
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train_jm.csv')

In [6]:
train.head(2)

,ID,first_party,second_party,facts,issued_area,sen_len,word_len,first_party_winner,first_party_win_percent,first_party_ner,second_party_ner,first_party1,second_party1,facts1
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",others,7,201,1,0.5,PERSON,PERSON,Phil|A.|St.|Amant,Herman|A.|Thompson,"On June 27, 1962, <p1>a candidate for public o..."
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,criminal,7,219,0,0.5,PERSON,PERSON,Stephen|Duncan,Lawrence|Owens,Ramon Nelson was riding his bike when he suffe...


In [7]:
test.head(2)

,ID,first_party,second_party,facts,issued_area,sen_len,word_len,first_party_win_percent,second_party_ner,first_party_ner,first_party1,second_party1,facts1
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...,others,2,55,0.5,GPE,others,Salerno,United|States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...,criminal,7,209,0.5,ORG,PERSON,Milberg|Weiss|Bershad|Hynes|and|Lerach,"Lexecon,|Inc.",Lexecon <p2>was a defendant in a class action ...


train, test split

In [41]:
import random

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [43]:
train_df['winner']=0
train_df['winner_frequency']=0
train_df['win_percentage']=0

for i in range(len(train_df)):
  if train_df['first_party_winner'][i]==1:
    train_df['winner'][i] = train_df['first_party'][i]
  else:
    train_df['winner'][i] = train_df['second_party'][i]

train_df['first_party_win_percent'] = 0
train_df['first_party_frequency'] = 0

for i in range(len(train_df)):
  train_df['first_party_frequency'][i] = len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])

  if (len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]])) > 1:
    train_df['first_party_win_percent'][i]= len(train_df.loc[train_df['winner'] == train_df['first_party'][i]]) /(len(train_df.loc[train_df['first_party'] == train_df['first_party'][i]]) + len(train_df.loc[train_df['second_party'] == train_df['first_party'][i]]))
  else:
    train_df['first_party_win_percent'][i] = 0.5

train_df=train_df.drop(['winner',	'winner_frequency','first_party_frequency','win_percentage'],axis=1)

In [44]:
val_df_first_party = val_df['first_party']
train_df_first_party = train_df[['first_party','first_party_win_percent']]
train_df_first_party=train_df_first_party.drop_duplicates()
test_first_party_병합 = pd.merge(val_df_first_party, train_df_first_party, how='left')
test_first_party_병합 = test_first_party_병합.fillna(0.5)
val_df['first_party_win_percent'] = test_first_party_병합['first_party_win_percent']

In [45]:
train_df.columns

Index(['ID', 'first_party', 'second_party', 'facts', 'issued_area', 'sen_len',
       'word_len', 'first_party_winner', 'first_party_win_percent',
       'first_party_ner', 'second_party_ner', 'first_party1', 'second_party1',
       'facts1'],
      dtype='object')

In [47]:
val_df.columns

Index(['ID', 'first_party', 'second_party', 'facts', 'issued_area', 'sen_len',
       'word_len', 'first_party_winner', 'first_party_win_percent',
       'first_party_ner', 'second_party_ner', 'first_party1', 'second_party1',
       'facts1'],
      dtype='object')

In [48]:
def categorize_probability(x):
    if x == 0:
        return 0
    elif 0 < x <= 0.33:
        return 1
    elif 0.33 < x < 0.5:
        return 2
    elif x == 0.5:
        return 3
    elif 0.5 < x < 0.57:
        return 2
    elif 0.57 <= x < 0.75:
        return 4
    elif 0.75 <= x < 1:
        return 5
    elif x == 1:
        return 6
    else:
        return -1

In [49]:
train_df['승률범주형분류'] = 0

In [50]:
for i in range(len(train_df)):
  train_df['승률범주형분류'][i] = categorize_probability(train_df['first_party_win_percent'][i])

In [51]:
val_df['승률범주형분류'] = 0

In [52]:
for i in range(len(val_df)):
  val_df['승률범주형분류'][i] = categorize_probability(val_df['first_party_win_percent'][i])

In [37]:
from nltk.corpus import names #corpus=말뭉치,이름 관련 부분 다루기 위한 객체


In [38]:
# 영어 데이터 전처리 함수
stops = set(stopwords.words('english'))
ps = nltk.stem.porter.PorterStemmer()
all_names=set(names.words())
lem = nltk.stem.wordnet.WordNetLemmatizer()

def cleaning(str):
    replaceAll = str

    # 특수문자 및 기호 등 필요없는 문자 제거
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)

    # 대소문자 모두 소문자로 통일
    no_capitals = only_english.lower().split()

    # 이름, 불용어(분석에 필요없는 토큰) 제거
    all_names=set(names.words())
    no_stops = [word for word in no_capitals if not word in all_names|stops]

    # 어근 추츨을 통한 텍스트 정규화 작업
    stemmer_words = [ps.stem(word) for word in no_stops]

    # 표제어 추출 Lemmatisation (convert the word into root word)
    lem_text = [lem.lemmatize(word) for word in stemmer_words]

    # back to string from list
    text = " ".join(lem_text)

    return text

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [116]:
vectorizer = TfidfVectorizer(max_features = 400)

def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['fact_processing'])
    else:
        X_facts = vectorizer.transform(df['fact_processing'])

    X = np.concatenate([X_facts.todense()], axis=1)
    return X

In [54]:
# 데이터 클리닝
train_df["fact_processing"] = train_df["facts"].apply(cleaning)
#train_fact = train_df["fact_processing"]

val_df["fact_processing"] = val_df["facts"].apply(cleaning)
#test_fact = test_df["fact_processing"]

In [61]:
train_df['first_party_ner'].astype("category")
train_df['second_party_ner'].astype("category")
train_df['issued_area'].astype("category")

val_df['first_party_ner'].astype("category")
val_df['second_party_ner'].astype("category")
val_df['issued_area'].astype("category")

0         civil
1        others
2        others
3        others
4        others
         ...   
491      others
492      others
493      others
494      others
495    criminal
Name: issued_area, Length: 496, dtype: category
Categories (3, object): ['civil', 'criminal', 'others']

In [83]:
# 특정 범주형 변수(categorical variable)을 더미변수 생성
train_cat = pd.get_dummies(data = train_df[["first_party_ner","second_party_ner","issued_area"]])
val_cat = pd.get_dummies(data = val_df[["first_party_ner","second_party_ner","issued_area"]])

In [117]:
# 데이터 벡터화
train_fact = get_vector(vectorizer, train_df, True)
train_fact = np.asarray(train_fact)
train_fact = pd.DataFrame(data=train_fact)

val_fact = get_vector(vectorizer, val_df, False)
val_fact = np.asarray(val_fact)
val_fact = pd.DataFrame(data=val_fact)

In [93]:
train_num = train_df['승률범주형분류']
train_target = train_df[['first_party_winner']]

val_num = val_df['승률범주형분류']

In [118]:
train_fin = pd.concat([train_fact,train_cat,train_num,train_target],axis=1,join='inner')
val_fin = pd.concat([val_fact, val_cat, val_num],axis=1,join='inner')

In [119]:
X_train = train_fin.drop(columns=['first_party_winner'])
y_train = train_fin['first_party_winner']

X_test = val_fin

In [120]:
y_test = val_df['first_party_winner']

In [121]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [122]:
model = LogisticRegression()

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [123]:
model.score(X_test,y_test)

0.6310483870967742

In [124]:
pred = model.predict(X_test)
pd.DataFrame(pred).value_counts()

1    423
0     73
dtype: int64

랜덤포레스트

In [125]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [126]:
model.score(X_test,y_test)

0.6491935483870968

KNeighborsClassifier(n_neighbors=3)

In [127]:
model = knn=KNeighborsClassifier(n_neighbors=7)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [128]:
model.score(X_test,y_test)

0.5766129032258065

CATBOOST

In [ ]:
!pip install catboost

In [111]:
from catboost import CatBoostClassifier, Pool

In [112]:
counts = list(y_train.value_counts())
class_weight = [counts[1]/sum(counts), counts[0]/sum(counts)]
print("weight :", class_weight)

weight : [0.33350151362260344, 0.6664984863773966]


In [129]:
model = CatBoostClassifier(random_seed=42,class_weights=class_weight, verbose=0)
model.fit(X_train, y_train)

In [130]:
model.score(X_test,y_test)

0.6592741935483871

In [131]:
pred = model.predict(X_test)
pd.DataFrame(pred).value_counts()

1    489
0      7
dtype: int64

#**party 종류 넣을까말까 정하기**